In [17]:
import folium
import zipcodes
import pandas as pd
from datetime import datetime

In [18]:
# create map view centered on St Louis County
m = folium.Map(location=(38.64, -90.44), zoom_start=10)

In [19]:
# load hotness data based on zipcode from realtor.com
df = pd.read_csv('RDC_Inventory_Hotness_Metrics_Zip_History.csv')

# clean data where zip_name is nan, we cannot extract state from these
df.dropna(subset=['zip_name'], inplace=True)

# extract state and convert numerical date into timestamp
df['state'] = df.apply(lambda x: x['zip_name'].split(', ')[-1].upper(), axis=1)
df['period'] = df.apply(lambda x:  datetime.strptime(str(x['month_date_yyyymm']), '%Y%m'), axis=1)

# filter by state="MO" (Missouri) and latest date
df_filter = df[(df['state']=='MO') & (df['period']==df['period'].max())] 

In [20]:
# calculate county averages, sort, and assign colors
counties = {'Jefferson County':[], 'St. Louis County':[], 'St. Louis city':[], 'Franklin County':[], 'Lincoln County':[], 'St. Charles County':[], 'Warren County':[]}
for index, row in df_filter.iterrows():
    z = zipcodes.matching(str(row['postal_code']))[0]
    if z['county'] in counties.keys():
        counties[z['county']].append(row['hotness_rank'])

county_avg = {k:sum(v)/len(v) for k,v in counties.items()}
sorted_items = sorted(county_avg.items(), key=lambda item: item[1])
county_sorted = dict(sorted_items)
county_color = {k:c for k,c in zip(county_sorted.keys(),['red','orange','beige','green','cadetblue','blue','purple'])}

In [21]:
# create zipcode markers with appropriate colors for county
for index, row in df_filter.iterrows():
    z = zipcodes.matching(str(row['postal_code']))[0]
    if z['county'] in counties.keys():
        folium.Marker(
            location=[z['lat'],z['long']],
            tooltip=f"zip: {z['zip_code']} rank: {row['hotness_rank']}",
            popup=f"{z['city']},\n{z['county']}",
            icon=folium.Icon(color=county_color[z['county']]),
        ).add_to(m)

In [26]:
# create legend for map using html
legend_html = '''
    <div style="position: fixed;
    bottom: 50px; right: 50px; width: 250px; height: 75px;
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white; opacity: 0.85;">
        &nbsp;<b>Hotness By County</b>
        <br><br>
        <div style="display:flex;">
            &nbsp;Hottest&nbsp;
            <div style="background-color: red; height:20px; width: 20px"></div>
            <div style="background-color: orange; height:20px; width: 20px"></div>
            <div style="background-color: beige; height:20px; width: 20px"></div>
            <div style="background-color: green; height:20px; width: 20px"></div>
            <div style="background-color: cadetblue; height:20px; width: 20px"></div>
            <div style="background-color: blue; height:20px; width: 20px"></div>
            <div style="background-color: purple; height:20px; width: 20px"></div>
            &nbsp;Coldest
        </div>
    </div>
'''
output = m.get_root().html.add_child(folium.Element(legend_html))

In [27]:
m